In [152]:
import pandas as pd
from geopy.geocoders import Nominatim
from geopy.distance import geodesic
import time

In [ ]:
# input_path = "../data/ortho_programs_full.csv"
# output_path = "../data/ortho_programs_cleaned.csv"

# with open(input_path, "r") as infile, open(output_path, "w") as outfile:
#     for i, line in enumerate(infile):
#         # Keep the header as-is
#         if i == 0:
#             outfile.write(line.strip() + "\n")
#             continue

#         # Split only on the first comma
#         parts = line.strip().split(",", 1)

#         if len(parts) == 2:
#             name = parts[0].strip()
#             address = parts[1].strip()
#             # Add quotes around the name if not already quoted
#             if not name.startswith('"'):
#                 name = f'"{name}"'
#             outfile.write(f"{name},{address}\n")
#         else:
#             print(f"Skipping malformed line {i + 1}: {line}")

In [ ]:
# ========== STEP 1: Load your data ==========
law_df = pd.read_csv("../data/law_schools.csv")  # columns: Name, Full Address
#ortho_df_short = pd.read_csv("../data/ortho_programs.csv")  # columns: Name, Full Address
ortho_df= pd.read_csv("../data/ortho_programs_cleaned.csv")  # columns: Name, Full Address

In [89]:
#law_df
ortho_df

,Name,Full Address
0,1. Hospital for Special Surgery/Cornell Medica...,535 East 70th Street New York NY 10021
1,2. Mayo Clinic College of Medicine and Science...,"200 First Street, Rochester MN 55905"
2,3. NYU Grossman School of Medicine/NYU Langone...,"301 East 17th Street, New York, NY 10003"
3,4. Washington University,"4455 Duncan Avenue, St. Louis, MO 63110"
4,5. Mass General Brigham/Massachusetts General ...,"55 Fruit Street, Boston, MA 02114"
...,...,...
202,203. Carilion Clinic Virginia Tech Carilion Sc...,"1 Riverside Circle, Roanoke, VA 24016"
203,204. Garden City Hospital Orthopaedic Surgery ...,"6245 Inkster Rd, Garden City, MI 48135"
204,205. Larkin Community Hospital Orthopaedic Sur...,"7000 SW 62nd Ave, South Miami, FL 33143"
205,206. St Joseph's Medical Center Orthopaedic Su...,"1800 N California St, Stockton, CA 95204"


In [101]:
# ========== STEP 2: Geocode each address ==========
geolocator = Nominatim(user_agent="law_ortho_matcher")

def geocode_address(address):
    for _ in range(3):  # retry in case of rate limit
        try:
            location = geolocator.geocode(address)
            if location:
                return pd.Series([location.latitude, location.longitude])
        except Exception:
            time.sleep(1)
    return pd.Series([None, None])

print("Geocoding law schools...")
law_df[["Lat", "Lon"]] = law_df["Full Address"].apply(geocode_address)
print("Done!")

Geocoding law schools...
Done!


In [90]:

print("Geocoding ortho programs...")
ortho_df[["Lat", "Lon"]] = ortho_df["Full Address"].apply(geocode_address)
print("Done!")

Geocoding ortho programs...
Done!


In [102]:
ortho_df.to_csv("../data/geo/ortho_lat_long.csv")
law_df.to_csv("../data/geo/law_lat_long.csv")

In [ ]:
# Define region rules based on lat/lon bounding boxes or city keywords
def get_region(lat, lon, name):
    name_lower = name.lower()
    
    if "new york" in name_lower or "rutgers" in name_lower or "nyu" in name_lower:
        return "NYC area"
    if "boston" in name_lower or "mass general" in name_lower or "harvard" in name_lower or "tufts" in name_lower:
        return "Boston"
    if "los angeles" in name_lower or "ucla" in name_lower or "cedars" in name_lower:
        return "LA"
    if "san francisco" in name_lower or "stanford" in name_lower or "ucsf" in name_lower:
        return "San Francisco"
    if "texas" in name_lower or "austin" in name_lower or "houston" in name_lower or "dallas" in name_lower:
        return "Texas"
    if "north carolina" in name_lower or "chapel hill" in name_lower or "duke" in name_lower or "charlotte" in name_lower:
        return "NC"
    if "charleston" in name_lower or "virginia" in name_lower or "uva" in name_lower:
        return "South (not NC)"
    if "chicago" in name_lower or "northwestern" in name_lower or "michigan" in name_lower or "ann arbor" in name_lower:
        return "Chicago / Michigan"
    if "washington" in name_lower or "oregon" in name_lower or "seattle" in name_lower:
        return "West"
    return "Other"



Boston
  49. Boston University Medical Center Orthopaedic Surgery Residency
  5. Mass General Brigham/Massachusetts General Hospital/Brigham and Women's Hospital/Harvard Medical School (Boston)
  56. Tufts Medical Center Orthopaedic Surgery Residency

Chicago / Michigan
  21. University of Michigan
  27. McGaw Medical Center of Northwestern University
  59. University of Chicago Orthopaedic Surgery Residency

LA
  40. UCLA David Geffen School of Medicine/UCLA Medical Center Orthopaedic Surgery Residency
  47. Cedars Sinai Medical Center Orthopaedic Surgery Residency

NC
  7. Duke Orthopaedic Surgery Residency
  72. University of North Carolina Hospitals Orthopaedic Surgery Residency

NYC area
  20. New York Presbyterian Hospital (Columbia Campus)
  3. NYU Grossman School of Medicine/NYU Langone Orthopedic Hospital

Other
  1. Hospital for Special Surgery/Cornell Medical Center
  11. Sidney Kimmel Medical College at Thomas Jefferson University/TJUH
  18. University of Pennsylvania Heal

## Restart from Here

In [153]:
ortho_df = pd.read_csv("../data/geo/ortho_lat_long.csv")
law_df= pd.read_csv("../data/geo/law_lat_long.csv")

In [154]:
ortho_df

,Unnamed: 0,Name,Full Address,Lat,Lon
0,0,1. Hospital for Special Surgery/Cornell Medica...,535 East 70th Street New York NY 10021,40.765164,-73.952690
1,1,2. Mayo Clinic College of Medicine and Science...,"200 First Street, Rochester MN 55905",43.170884,-77.588008
2,2,3. NYU Grossman School of Medicine/NYU Langone...,"301 East 17th Street, New York, NY 10003",40.734394,-73.982991
3,3,4. Washington University,"4455 Duncan Avenue, St. Louis, MO 63110",38.637262,-90.257881
4,4,5. Mass General Brigham/Massachusetts General ...,"55 Fruit Street, Boston, MA 02114",42.362860,-71.068753
...,...,...,...,...,...
202,202,203. Carilion Clinic Virginia Tech Carilion Sc...,"1 Riverside Circle, Roanoke, VA 24016",37.256684,-79.942362
203,203,204. Garden City Hospital Orthopaedic Surgery ...,"6245 Inkster Rd, Garden City, MI 48135",42.332312,-83.313989
204,204,205. Larkin Community Hospital Orthopaedic Sur...,"7000 SW 62nd Ave, South Miami, FL 33143",25.705536,-80.293993
205,205,206. St Joseph's Medical Center Orthopaedic Su...,"1800 N California St, Stockton, CA 95204",37.968972,-121.288189


## How Important the Rank of Law School is vs Ortho Rank 
higher number penalizes distance more as this is a minimization problem

In [155]:
law_rank_importance = 5
ortho_rank_importance= 1
dist_rank_importance= 5
culture_weight= 10

In [160]:


# 2) add a CultureScore column from a dict (0..1 scale). Missing = 0.
ortho_culture = {
    "5. Mass General Brigham/Massachusetts General Hospital/Brigham and Women's Hospital/Harvard Medical School (Boston)": 3,  # x boost
    "20. New York Presbyterian Hospital (Columbia Campus)":-3,
    "3. NYU Grossman School of Medicine/NYU Langone Orthopedic Hospital":-1,
    "1. Hospital for Special Surgery/Cornell Medical Center":-1,
    "72. University of North Carolina Hospitals Orthopaedic Surgery Residency":5,
    "7. Duke Orthopaedic Surgery Residency":2,
    "16. Carolinas Medical Center":3,
    "71. Dell Medical School at UT Austin Orthopaedic Surgery Residency":5,
    "65. Prisma Health/University of South Carolina SOM Greenville Orthopaedic Surgery Residency":5,
    "18. University of Pennsylvania Health System":-2,
    "11. Sidney Kimmel Medical College at Thomas Jefferson University/TJUH":-2
    # add more here...
}

law_culture = {
    # e.g. "7. Harvard Law School": 0.25,
    "7. Harvard Law School":3,
    "15. University of Texas School of Law":1
}



# assuming ortho_df and law_df already exist with Name/Lat/Lon/etc.
import pandas as pd

ortho_df["CultureScore"] = ortho_df["Name"].map(ortho_culture).fillna(0.0)
law_df["CultureScore"]   = law_df["Name"].map(law_culture).fillna(0.0)

# 3) use it in your scoring (subtract because you minimize total score)
def pair_score(law, ortho, distance):
    law_rank   = int(law["Name"].split(".")[0])
    ortho_rank = int(ortho["Name"].split(".")[0])
    return (
        law_rank_importance   * law_rank +
        ortho_rank_importance * ortho_rank +
        dist_rank_importance  * distance
        - culture_weight * (float(law.get("CultureScore", 0)) + float(ortho.get("CultureScore", 0)))
    )

# 4) your loop (unchanged except calling pair_score and using culture already on the DFs)
matches = []
for _, law in law_df.iterrows():
    for _, ortho in ortho_df.iterrows():
        if pd.notnull(law["Lat"]) and pd.notnull(ortho["Lat"]):
            distance = geodesic((law["Lat"], law["Lon"]), (ortho["Lat"], ortho["Lon"])).miles
            score = pair_score(law, ortho, distance)
            matches.append({
                "Law School": law["Name"],
                "Ortho Program": ortho["Name"],
                "Distance (miles)": round(distance, 2),
                "Score": round(score, 3)
            })

In [ ]:
# # ========== STEP 3: Match and score all pairwise combos ==========
# matches = []
# for _, law in law_df.iterrows():
#     for _, ortho in ortho_df.iterrows():
#         if pd.notnull(law["Lat"]) and pd.notnull(ortho["Lat"]):
#             distance = geodesic((law["Lat"], law["Lon"]), (ortho["Lat"], ortho["Lon"])).miles
#             law_rank = int(law["Name"].split(".")[0])
#             ortho_rank = int(ortho["Name"].split(".")[0])
            
#             # Example scoring weights (adjust as needed)
#             score = law_rank_importance * law_rank + ortho_rank_importance * ortho_rank + dist_rank_importance * distance

#             matches.append({
#                 "Law School": law["Name"],
#                 "Ortho Program": ortho["Name"],
#                 "Distance (miles)": round(distance, 2),
#                 "Score": round(score, 2)
#             })


In [161]:
# ========== STEP 4: Sort and get top matches ==========
results_df = pd.DataFrame(matches).sort_values("Score").reset_index(drop=True)

# Save or display
results_df.to_csv("../results/top_law_ortho_matches.csv", index=False)
print("Top 30 matches:")
print(results_df.head(30))



Top 30 matches:
                                        Law School  \
0                            7. Harvard Law School   
1                 6. Duke University School of Law   
2                           1. Stanford Law School   
3              3. University of Chicago Law School   
4             8. New York University School of Law   
5          4. University of Virginia School of Law   
6   5. University of Pennsylvania Carey Law School   
7   5. University of Pennsylvania Carey Law School   
8             8. New York University School of Law   
9             9. University of Michigan Law School   
10                              2. Yale Law School   
11                           7. Harvard Law School   
12                         10. Columbia Law School   
13             3. University of Chicago Law School   
14         11. Northwestern Pritzker School of Law   
15                           7. Harvard Law School   
16             3. University of Chicago Law School   
17         1

In [162]:
ortho_signals= 30
law_signals = 25

# Get top 30 unique orthopedic surgery programs
top_ortho_programs = []
seen_ortho = set()
for _, row in results_df.iterrows():
    ortho = row['Ortho Program']
    if ortho not in seen_ortho:
        top_ortho_programs.append(ortho)
        seen_ortho.add(ortho)
    if len(top_ortho_programs) == ortho_signals:
        break

# Get top 25 unique law schools
top_law_schools = []
seen_law = set()
for _, row in results_df.iterrows():
    law = row['Law School']
    if law not in seen_law:
        top_law_schools.append(law)
        seen_law.add(law)
    if len(top_law_schools) == law_signals:
        break

    

# Now top_ortho_programs and top_law_schools are your filtered lists
print("Top 30 unique ortho programs:")
for o in top_ortho_programs:
    print(o)

print("\nTop 25 unique law schools:")
for l in top_law_schools:
    print(l)

Top 30 unique ortho programs:
5. Mass General Brigham/Massachusetts General Hospital/Brigham and Women's Hospital/Harvard Medical School (Boston)
7. Duke Orthopaedic Surgery Residency
23. Stanford Orthopedic Residency Program
6. Rush Medical Center Orthopaedic Surgery Residency
3. NYU Grossman School of Medicine/NYU Langone Orthopedic Hospital
31. University of Virginia Orthopaedic Surgery Residency
18. University of Pennsylvania Health System
11. Sidney Kimmel Medical College at Thomas Jefferson University/TJUH
1. Hospital for Special Surgery/Cornell Medical Center
21. University of Michigan
60. Yale-New Haven Medical Center Orthopaedic Surgery Residency
49. Boston University Medical Center Orthopaedic Surgery Residency
59. University of Chicago Orthopaedic Surgery Residency
56. Tufts Medical Center Orthopaedic Surgery Residency
27. McGaw Medical Center of Northwestern University
71. Dell Medical School at UT Austin Orthopaedic Surgery Residency
8. Vanderbilt Orthopaedic Surgery Resid

In [163]:
top_ortho_df = pd.DataFrame(top_ortho_programs)
# Now you can save it to CSV
top_ortho_df.to_csv("../results/ortho_signals.csv", index=False)

In [ ]:

# Apply to the top 30 ortho programs
top_ortho_df = pd.DataFrame(top_ortho_programs, columns=["Ortho Program"])
top_ortho_df = top_ortho_df.merge(ortho_df[["Name", "Lat", "Lon"]], left_on="Ortho Program", right_on="Name", how="left")
top_ortho_df["Region"] = top_ortho_df.apply(lambda row: get_region(row["Lat"], row["Lon"], row["Ortho Program"]), axis=1)

# Sort within regions
top_ortho_df = top_ortho_df.sort_values(["Region", "Ortho Program"])

# Output grouped
for region, group in top_ortho_df.groupby("Region"):
    print(f"\n{region}")
    for prog in group["Ortho Program"]:
        print("  " + prog)

# Save CSV if needed
top_ortho_df.to_csv("../results/ortho_signals_grouped.csv", index=False)